In [1]:
import gc
import os
import sys
import uproot
import pandas
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint, CSVLogger
from tqdm import tqdm_notebook
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
import functools

sys.path.insert(0, "../python")
from common import *
from WeightManager import WeightManager

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print(len(input_branches))
input_branches

181


['pt',
 'eta',
 'mass',
 'decayMode',
 'dxy',
 'dxy_sig',
 'dz',
 'ip3d',
 'ip3d_sig',
 'hasSecondaryVertex',
 'flightLength_r',
 'flightLength_dEta',
 'flightLength_dPhi',
 'flightLength_sig',
 'chargedIsoPtSum',
 'chargedIsoPtSumdR03',
 'footprintCorrection',
 'footprintCorrectiondR03',
 'neutralIsoPtSum',
 'neutralIsoPtSumdR03',
 'neutralIsoPtSumWeight',
 'neutralIsoPtSumWeightdR03',
 'photonPtSumOutsideSignalCone',
 'photonPtSumOutsideSignalConedR03',
 'puCorrPtSum',
 'pt_weighted_deta_strip',
 'pt_weighted_dphi_strip',
 'pt_weighted_dr_signal',
 'pt_weighted_dr_iso',
 'leadingTrackNormChi2',
 'e_ratio',
 'gj_angle_diff',
 'n_photons',
 'emFraction',
 'inside_ecal_crack',
 'has_gsf_track',
 'gsf_ele_matched',
 'gsf_ele_pt',
 'gsf_ele_dEta',
 'gsf_ele_dPhi',
 'gsf_ele_energy',
 'gsf_ele_Ee',
 'gsf_ele_Egamma',
 'gsf_ele_Pin',
 'gsf_ele_Pout',
 'gsf_ele_Eecal',
 'gsf_ele_dEta_SeedClusterTrackAtCalo',
 'gsf_ele_dPhi_SeedClusterTrackAtCalo',
 'gsf_ele_mvaIn_sigmaEtaEta',
 'gsf_ele_mvaI

In [3]:
input_file = 'file://C:/Users/konst/workspace/tau-ml/tuples/training_v4/all_shuffled.root'
weight_file = 'C:/Users/konst/workspace/tau-ml/tuples/training_v4/all_shuffled_weights.h5'

In [4]:
X, Y = ReadXY(input_file, 'taus')#, entrystop=int(7e5))
X.shape, Y.shape

((65452708, 181), (65452708, 4))

In [5]:
weightManager = WeightManager(weight_file, calc_weights=False)
#weightManager = WeightManager('20L1024N50D_relu_0723_s1_last_pass_weights.hdf5', calc_weights=False, first_block=False)

In [6]:
class ClassSFManager:
    def __init__(self, n_items, init_sf):
        self.sf = np.array(init_sf, dtype=float)
        self.sf_inputs = np.empty((n_items, len(init_sf)), dtype=float)
        self.UpdateInputs()
        
    def Update(self, fit_hist):
        loss_e = fit_hist.history['Le'][-1]
        loss_mu = fit_hist.history['Lmu'][-1]
        loss_jet = fit_hist.history['Ljet'][-1]
        
        print("Losses: ", loss_e, loss_mu, loss_jet)
        print("History: ", fit_hist.history)
        
        self.sf[0] = 1
        self.sf[1] = loss_e / loss_mu * 0.9
        self.sf[2] = loss_e / loss_jet * 1.1
        self.UpdateInputs()
        
    def UpdateInputs(self):
        for n in range(len(self.sf)):
            self.sf_inputs[:, n] = self.sf[n]

In [7]:
cl_sf_manager = ClassSFManager(X.shape[0], [1, 2, 5])

In [2]:
def create_model(n_hidden_layers, n_neurons, dropout_rate, activation):
    if activation == 'relu':
        kernel_init = 'he_uniform'
        DropoutType = Dropout
        apply_batch_norm = True
    elif activation == 'selu':
        kernel_init = 'lecun_normal'
        DropoutType = AlphaDropout
        apply_batch_norm = False
    else:
        raise RuntimeError('Activation "{}" is not supported'.format(activation))
    
    model_name = "{}L{}N{}D_{}".format(n_hidden_layers, n_neurons, int(dropout_rate*100), activation)
    input_layer = Input(name="main_input", shape=input_shape)
    
    prev_layer = input_layer
    for n in range(n_hidden_layers):
        dense_layer = Dense(n_neurons, name="dense_%d" % n, kernel_initializer=kernel_init)(prev_layer)
        layer = dense_layer
        if apply_batch_norm:
            layer = BatchNormalization(name="batch_norm_%d" % n)(dense_layer)
        activation_layer = Activation(activation, name="activation_%d" % n)(layer)
        prev_layer = DropoutType(dropout_rate, name="dropout_%d" % n)(activation_layer)
    output_layer = Dense(n_outputs, name="dense_%d" % n_hidden_layers)(prev_layer)
    softmax_output = Activation("softmax", name="main_output")(output_layer)

    weight_input = Input(name="weight_input", shape=(3,))
    sf_input = Input(name="sf_input", shape=(3,))
    model = Model([input_layer, weight_input, sf_input], softmax_output, name="deepTauId")
    return model, model_name

In [3]:
def compile_model(model, learning_rate):
    #opt = keras.optimizers.Adam(lr=learning_rate)
    opt = keras.optimizers.Nadam(lr=learning_rate)
    #opt = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    
    weight_input = model.inputs[1]
    sf_input = model.inputs[2]
    tau_crossentropy = functools.partial(TauLosses.tau_crossentropy, weights=weight_input, sf=sf_input)
    Le = functools.partial(TauLosses.Le, weights=weight_input)
    Lmu = functools.partial(TauLosses.Lmu, weights=weight_input)
    Ljet = functools.partial(TauLosses.Ljet, weights=weight_input)
    sLe = functools.partial(TauLosses.sLe, weights=weight_input, sf=sf_input)
    sLmu = functools.partial(TauLosses.sLmu, weights=weight_input, sf=sf_input)
    sLjet = functools.partial(TauLosses.sLjet, weights=weight_input, sf=sf_input)
    
    functools.update_wrapper(tau_crossentropy, TauLosses.tau_crossentropy)
    functools.update_wrapper(Le, TauLosses.Le)
    functools.update_wrapper(Lmu, TauLosses.Lmu)
    functools.update_wrapper(Ljet, TauLosses.Ljet)
    functools.update_wrapper(sLe, TauLosses.sLe)
    functools.update_wrapper(sLmu, TauLosses.sLmu)
    functools.update_wrapper(sLjet, TauLosses.sLjet)
    
    model.compile(loss=tau_crossentropy, optimizer=opt, metrics=["accuracy", Le, Lmu, Ljet, sLe, sLmu, sLjet])

In [4]:
model, model_name = create_model(20, 1024, 0.5, 'relu')
#model, model_name = create_model(20, 1024, 0.5, 'selu')

#model.load_weights("20L1024N50D_relu_0723_s1_final.hdf5", by_name=True)

compile_model(model, 1e-3)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 181)               0         
_________________________________________________________________
dense_0 (Dense)              (None, 1024)              186368    
_________________________________________________________________
batch_norm_0 (BatchNormaliza (None, 1024)              4096      
_________________________________________________________________
activation_0 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_0 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 1024)              4096      
__________

_________________________________________________________________
dense_15 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_norm_15 (BatchNormaliz (None, 1024)              4096      
_________________________________________________________________
activation_15 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_norm_16 (BatchNormaliz (None, 1024)              4096      
_________________________________________________________________
activation_16 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_16

In [11]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [13]:
with keras.backend.get_session().as_default() as sess:
    graph = sess.graph
    tf.contrib.quantize.create_training_graph(input_graph=graph)
    graph_def = sess.graph.as_graph_def()

In [12]:
def run_training(train_suffix, model_name, epoch, n_pass):
    n_train = int(6e7)
    n_test = X.shape[0] - n_train
    n_per_step = int(1e7)

    train_name = '%s_%s' % (model_name, train_suffix)
    
    weightManager.SetHistFileName('%s_weight_upd.hdf5' % train_name)
    
    cb_acc = ModelCheckpoint("%s_acc.hdf5" % train_name, monitor="val_acc", save_best_only=True,
                             save_weights_only=False, mode="max", verbose=1)
    
    cb_losses = []
    for loss_name in ["loss", "Le", "Lmu", "Ljet"]:
        cb_losses.append(ModelCheckpoint("%s_%s.hdf5" % (train_name, loss_name), monitor="val_%s" % loss_name,
                                         save_best_only=True, save_weights_only=False, mode="min", verbose=1))

    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    for pass_id in range(0, n_pass):
        print("Starting pass %s" % pass_id)
        for pos in range(0, n_train, n_per_step):
            if epoch != 0:
                print("Updating weights...")
                weightManager.UpdateWeights(model, epoch, X, pos, n_per_step, cl_sf_manager.sf_inputs,
                                            [('jet', 0.6), ('e', 0.6), ('mu', 0.9)])
            
            val_data = ([X[n_train:n_train+n_test, :], weightManager.GetWeights(n_train, n_train+n_test),
                         cl_sf_manager.sf_inputs[n_train:n_train+n_test, :]],
                        Y[n_train:n_train+n_test, :])
            pbar = TQDMNotebookCallback(leave_outer=False, show_outer=False, leave_inner = True)
            callbacks = [pbar, csv_log, cb_acc, *cb_losses]
            fit_hist = model.fit([X[pos:pos+n_per_step, :], weightManager.GetWeights(pos, pos+n_per_step),
                                  cl_sf_manager.sf_inputs[pos:pos+n_per_step, :]],
                                 Y[pos:pos+n_per_step, :], callbacks=callbacks, validation_data=val_data, batch_size=10000,
                                 epochs=epoch+1, initial_epoch=epoch, verbose=0)
            epoch += 1
        if pass_id + 1 != n_pass:
            weightManager.SaveWeights('%s_last_pass_weights.hdf5' % train_name)
            model.save("%s_last_pass.hdf5" % train_name)
            if ((pass_id+1) % 10) == 0:
                weightManager.SaveWeights('%s_pass_%d_weights.hdf5' % (train_name, pass_id+1))
                model.save("%s_pass_%d.hdf5" % (train_name, pass_id+1))
                cl_sf_manager.Update(fit_hist)
                print("New class sf:", cl_sf_manager.sf)

    weightManager.SaveWeights('%s_final_weights.hdf5' % train_name)
    model.save("%s_final.hdf5" % train_name)

In [13]:
run_training('0807_s1', model_name, 0, 200)

Starting pass 0




Epoch 00001: val_acc improved from -inf to 0.49136, saving model to 20L1024N50D_relu_0807_s1_acc.hdf5

Epoch 00001: val_loss improved from inf to 4.79027, saving model to 20L1024N50D_relu_0807_s1_loss.hdf5

Epoch 00001: val_Le improved from inf to 0.70313, saving model to 20L1024N50D_relu_0807_s1_Le.hdf5

Epoch 00001: val_Lmu improved from inf to 0.70410, saving model to 20L1024N50D_relu_0807_s1_Lmu.hdf5

Epoch 00001: val_Ljet improved from inf to 0.53579, saving model to 20L1024N50D_relu_0807_s1_Ljet.hdf5
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 297, average sf = 1.9999993678427952
tau_vs_e: bins changed = 297, average sf = 1.9999993678427952
tau_vs_mu: bins changed = 106, average sf = 1.0343037658749816




Epoch 00002: val_acc improved from 0.49136 to 0.73400, saving model to 20L1024N50D_relu_0807_s1_acc.hdf5

Epoch 00002: val_loss improved from 4.79027 to 4.02956, saving model to 20L1024N50D_relu_0807_s1_loss.hdf5

Epoch 00002: val_Le improved from 0.70313 to 0.51981, saving model to 20L1024N50D_relu_0807_s1_Le.hdf5

Epoch 00002: val_Lmu improved from 0.70410 to 0.25384, saving model to 20L1024N50D_relu_0807_s1_Lmu.hdf5

Epoch 00002: val_Ljet did not improve from 0.53579
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 266, average sf = 1.0069126988928179
tau_vs_e: bins changed = 243, average sf = 1.0009121118054372
tau_vs_mu: bins changed = 22, average sf = 0.9130567458491509




Epoch 00003: val_acc did not improve from 0.73400

Epoch 00003: val_loss did not improve from 4.02956

Epoch 00003: val_Le did not improve from 0.51981

Epoch 00003: val_Lmu did not improve from 0.25384

Epoch 00003: val_Ljet did not improve from 0.53579
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 269, average sf = 1.0075626400290227
tau_vs_e: bins changed = 269, average sf = 1.0075820952668006
tau_vs_mu: bins changed = 93, average sf = 0.994458532920014




Epoch 00004: val_acc did not improve from 0.73400

Epoch 00004: val_loss improved from 4.02956 to 3.93673, saving model to 20L1024N50D_relu_0807_s1_loss.hdf5

Epoch 00004: val_Le did not improve from 0.51981

Epoch 00004: val_Lmu did not improve from 0.25384

Epoch 00004: val_Ljet improved from 0.53579 to 0.48132, saving model to 20L1024N50D_relu_0807_s1_Ljet.hdf5
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 255, average sf = 1.005706732985747
tau_vs_e: bins changed = 92, average sf = 1.004596562323561
tau_vs_mu: bins changed = 69, average sf = 0.9807955634200518


...



Epoch 01196: val_acc did not improve from 0.94969

Epoch 01196: val_loss did not improve from 0.40779

Epoch 01196: val_Le did not improve from 0.14037

Epoch 01196: val_Lmu did not improve from 0.01335

Epoch 01196: val_Ljet did not improve from 0.12076
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 167, average sf = 1.003840240814347
tau_vs_e: bins changed = 172, average sf = 1.002136243941342
tau_vs_mu: bins changed = 115, average sf = 1.02057715694868




Epoch 01197: val_acc did not improve from 0.94969

Epoch 01197: val_loss did not improve from 0.40779

Epoch 01197: val_Le did not improve from 0.14037

Epoch 01197: val_Lmu did not improve from 0.01335

Epoch 01197: val_Ljet did not improve from 0.12076
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 208, average sf = 1.0016439497204959
tau_vs_e: bins changed = 217, average sf = 1.0001918226541195
tau_vs_mu: bins changed = 184, average sf = 1.0082544570217957




Epoch 01198: val_acc did not improve from 0.94969

Epoch 01198: val_loss did not improve from 0.40779

Epoch 01198: val_Le did not improve from 0.14037

Epoch 01198: val_Lmu did not improve from 0.01335

Epoch 01198: val_Ljet did not improve from 0.12076
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 155, average sf = 0.9999311214197629
tau_vs_e: bins changed = 171, average sf = 1.000259631281977
tau_vs_mu: bins changed = 71, average sf = 0.999093368257705




Epoch 01199: val_acc did not improve from 0.94969

Epoch 01199: val_loss did not improve from 0.40779

Epoch 01199: val_Le did not improve from 0.14037

Epoch 01199: val_Lmu did not improve from 0.01335

Epoch 01199: val_Ljet did not improve from 0.12076
Updating weights...
	predictions has been calculated.
tau_vs_jet: bins changed = 183, average sf = 1.0035205280129662
tau_vs_e: bins changed = 163, average sf = 1.0006735660754076
tau_vs_mu: bins changed = 75, average sf = 1.0000325359318998




Epoch 01200: val_acc did not improve from 0.94969

Epoch 01200: val_loss did not improve from 0.40779

Epoch 01200: val_Le did not improve from 0.14037

Epoch 01200: val_Lmu did not improve from 0.01335

Epoch 01200: val_Ljet did not improve from 0.12076
